In [24]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras import layers, models

In [33]:
# Define path to the dataset
PATH = "C:/Users/pcost/OneDrive/Ambiente de Trabalho/UNI/MECD/2 Semestre/ACA/Projeto/ACA-Project/AML-Project2/dataset"

# Read the CSV file containing the image paths and labels
# Read the training set data
train_data = np.load(os.path.join(PATH, "trainX.npy"))
train_labels = np.load(os.path.join(PATH, "trainy.npy"), allow_pickle=True)


print(train_data.shape)
display(train_data)

# Read the test set data
test_data = tf.convert_to_tensor(test_data, dtype=tf.float32)

(12000, 50, 50, 3)


array([[[[ 47,  64,  77],
         [ 43,  57,  80],
         [182, 206, 224],
         ...,
         [ 10,   8,   8],
         [  9,   7,   7],
         [  7,   5,   5]],

        [[ 42,  59,  80],
         [ 14,  22,  35],
         [ 50,  62,  74],
         ...,
         [ 14,   9,  10],
         [ 10,   5,   6],
         [ 11,   6,   7]],

        [[ 51,  68,  87],
         [  0,   5,   9],
         [  7,  14,  23],
         ...,
         [ 13,   8,  10],
         [ 13,   8,   9],
         [ 11,   6,   7]],

        ...,

        [[ 57,  63,  76],
         [ 43,  55,  65],
         [ 33,  45,  63],
         ...,
         [ 37,  36,  40],
         [ 17,  12,  14],
         [ 16,  11,  13]],

        [[ 48,  58,  65],
         [ 47,  53,  66],
         [ 45,  53,  60],
         ...,
         [ 19,  21,  22],
         [ 25,  20,  22],
         [ 14,   9,  10]],

        [[ 32,  40,  47],
         [ 13,  19,  30],
         [ 11,  17,  24],
         ...,
         [237, 239, 247],
        

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_rem, y_train, y_rem = train_test_split(train_data, train_labels, train_size=0.6, random_state=42, stratify=train_labels)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=42, stratify=y_rem)

In [35]:
display(X_train.shape)

(7200, 50, 50, 3)

In [41]:
model = models.Sequential()

model.add(layers.Conv2D(32, 3, activation='relu', input_shape=(50, 50, 3)))
model.add(layers.Conv2D(64, 3, activation='relu'))
model.add(layers.Conv2D(128, 3, activation='relu'))
model.add(layers.MaxPool2D())
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_17 (Conv2D)          (None, 48, 48, 32)        896       
                                                                 
 conv2d_18 (Conv2D)          (None, 46, 46, 64)        18496     
                                                                 
 conv2d_19 (Conv2D)          (None, 44, 44, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 22, 22, 128)      0         
 2D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 22, 22, 128)       0         
                                                                 
 flatten_5 (Flatten)         (None, 61952)             0         
                                                      

In [42]:
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)


In [43]:
model.compile(
    optimizer='Adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=32,
    epochs=20,
    validation_split=0.25
)

Epoch 1/20
169/169 [==============================] - 45s 253ms/step - loss: 3.9596 - accuracy: 0.4815 - val_loss: 1.0549 - val_accuracy: 0.5844
Epoch 2/20
169/169 [==============================] - 50s 298ms/step - loss: 1.0381 - accuracy: 0.5861 - val_loss: 1.0249 - val_accuracy: 0.6028
Epoch 3/20
169/169 [==============================] - 56s 332ms/step - loss: 0.9811 - accuracy: 0.6081 - val_loss: 1.0256 - val_accuracy: 0.5972
Epoch 4/20
169/169 [==============================] - 68s 403ms/step - loss: 0.8699 - accuracy: 0.6502 - val_loss: 0.9032 - val_accuracy: 0.6422
Epoch 5/20
169/169 [==============================] - 58s 343ms/step - loss: 0.7627 - accuracy: 0.6944 - val_loss: 0.9514 - val_accuracy: 0.6333
Epoch 6/20
169/169 [==============================] - 63s 371ms/step - loss: 0.6675 - accuracy: 0.7393 - val_loss: 0.9842 - val_accuracy: 0.6544
Epoch 7/20
169/169 [==============================] - 63s 372ms/step - loss: 0.5620 - accuracy: 0.7865 - val_loss: 0.9295 - val_ac

In [45]:
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)

from sklearn.metrics import classification_report

model.evaluate(X_test, y_test)

pred = model.predict(X_test)
pred = np.argmax(pred, axis=1)

print(classification_report(y_test, pred, zero_division=0))
print('=========================================')

75/75 [==============================] - 5s 60ms/step
              precision    recall  f1-score   support

         0.0       0.77      0.64      0.70       400
         1.0       0.89      0.91      0.90       400
         2.0       0.59      0.81      0.68       400
         3.0       0.71      0.68      0.69       400
         4.0       0.73      0.55      0.63       400
         5.0       0.71      0.76      0.73       400

    accuracy                           0.72      2400
   macro avg       0.73      0.72      0.72      2400
weighted avg       0.73      0.72      0.72      2400



In [48]:
test_df = pd.read_csv(os.path.join(PATH, "test_intel-image-classification-csvdata-kaggle.csv"))

In [49]:
preds = model.predict(test_data)
predict_test = np.argmax(preds, axis=1)

submission = pd.DataFrame({'number': test_df['number'], 'class': predict_test})
print(submission)
submission.to_csv('submission_cnn.csv', index=False)

158/158 [==============================] - 9s 55ms/step
      number  class
0          0      5
1          1      5
2          2      2
3          3      0
4          4      2
...      ...    ...
5046    5046      1
5047    5047      0
5048    5048      5
5049    5049      5
5050    5050      5

[5051 rows x 2 columns]
